In [ ]:
import numpy as np
import pandas as pd
import os
import keras
from keras import models, layers

In [ ]:
data_dir = 'F:\study\ml\DataSet\jena_climate'
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')

f = open(fname)
data = f.read()
f.close()

In [3]:
lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]
print(header)
print(len(lines))

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420551


In [4]:
float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values

In [5]:
mean = float_data[:20000].mean(axis=0)
float_data -= mean
std = float_data[:20000].std(axis=0)
float_data /= std

In [6]:
float_data.shape

(420551, 14)

In [15]:
def generator(data,
              lookback,
              delay,
              min_index,
              max_index,
              shuffle=False,
              batch_size=128,
              step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback

    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index,
                                     batch_size)
        else:
            if i + batch_size > max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
#         print(rows)
#         print(len(rows))

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows), ))
        #         print(samples.shape)

        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            #             print(indices)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]


#             print(samples.shape)
        yield samples, targets

In [16]:
lookback = 1440
step = 6
delay = 144
batch_size = 128

In [17]:
train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step,
                      batch_size=batch_size)

In [18]:
# next(train_gen)

In [19]:
val_gen = generator(
    float_data,
    lookback=lookback,
    delay=delay,
    min_index=200001,
    max_index=300000,
    #                       shuffle=True,
    step=step,
    batch_size=batch_size)

In [20]:
val_steps = (300000 - 200001 - lookback) // batch_size

In [21]:
def evaluate_naive_method():
    batch_maes = []
    for step in range(val_steps):
        samples, targets = next(val_gen)
        preds = samples[:, -1, 1]
        mae = np.mean(np.abs(preds - targets))
        batch_maes.append(mae)
    print(np.mean(batch_maes))

In [22]:
evaluate_naive_method()

0.33551756286169604


In [23]:
from keras import models, layers, optimizers

In [24]:
model = models.Sequential()
model.add(layers.Flatten(input_shape=(lookback // step, float_data.shape[-1])))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))

In [25]:
model.compile(optimizer=optimizers.rmsprop(), loss='mae')
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=20,
                              validation_data=val_gen,
                              validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 11s 22ms/step - loss: 1.4615 - val_loss: 0.7871
Epoch 2/20
500/500 [==============================] - 11s 22ms/step - loss: 0.4892 - val_loss: 0.4033
Epoch 3/20
500/500 [==============================] - 11s 21ms/step - loss: 0.3409 - val_loss: 0.4125
Epoch 4/20
500/500 [==============================] - 11s 21ms/step - loss: 0.3018 - val_loss: 0.3961
Epoch 5/20
500/500 [==============================] - 11s 21ms/step - loss: 0.2848 - val_loss: 0.3653
Epoch 6/20
500/500 [==============================] - 11s 21ms/step - loss: 0.2729 - val_loss: 0.3872
Epoch 7/20
500/500 [==============================] - 11s 21ms/step - loss: 0.2659 - val_loss: 0.3758
Epoch 8/20
500/500 [==============================] - 11s 21ms/step - loss: 0.2580 - val_loss: 0.4109
Epoch 9/20
500/500 [==============================] - 11s 21ms/step - loss: 0.2522 - val_loss: 0.3944
Epoch 10/20
500/500 [==============================] - 11s 21ms/step - loss: 0.248

In [28]:
model2 = models.Sequential()
model2.add(layers.GRU(32, input_shape=(None, float_data.shape[-1])))
model2.add(layers.Dense(1))

In [29]:
model2.compile(optimizer=optimizers.rmsprop(), loss='mae')
history = model2.fit_generator(train_gen,
                               steps_per_epoch=500,
                               epochs=20,
                               validation_data=val_gen,
                               validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 59s 117ms/step - loss: 0.3678 - val_loss: 0.3180
Epoch 2/20
500/500 [==============================] - 65s 129ms/step - loss: 0.3285 - val_loss: 0.3090
Epoch 3/20
500/500 [==============================] - 72s 143ms/step - loss: 0.3183 - val_loss: 0.3081
Epoch 4/20
500/500 [==============================] - 70s 141ms/step - loss: 0.3070 - val_loss: 0.3113
Epoch 5/20
500/500 [==============================] - 66s 132ms/step - loss: 0.3028 - val_loss: 0.3192
Epoch 6/20
500/500 [==============================] - 80s 160ms/step - loss: 0.2971 - val_loss: 0.3235
Epoch 7/20
500/500 [==============================] - 59s 117ms/step - loss: 0.2918 - val_loss: 0.3218
Epoch 8/20
500/500 [==============================] - 65s 129ms/step - loss: 0.2853 - val_loss: 0.3284
Epoch 9/20
500/500 [==============================] - 59s 119ms/step - loss: 0.2808 - val_loss: 0.3304
Epoch 10/20
500/500 [==============================] - 64s 128ms/step - l

In [31]:
model3 = models.Sequential()
model3.add(
    layers.GRU(32,
               dropout=0.2,
               recurrent_dropout=0.2,
               input_shape=(None, float_data.shape[-1])))
model3.add(layers.Dense(1))

In [32]:
model3.compile(optimizer=optimizers.rmsprop(), loss='mae')
history = model3.fit_generator(train_gen,
                               steps_per_epoch=500,
                               epochs=20,
                               validation_data=val_gen,
                               validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 71s 141ms/step - loss: 0.3856 - val_loss: 0.3261
Epoch 2/20
500/500 [==============================] - 70s 139ms/step - loss: 0.3653 - val_loss: 0.3207
Epoch 3/20
500/500 [==============================] - 66s 132ms/step - loss: 0.3566 - val_loss: 0.3193
Epoch 4/20
500/500 [==============================] - 66s 132ms/step - loss: 0.3509 - val_loss: 0.3125
Epoch 5/20
500/500 [==============================] - 66s 131ms/step - loss: 0.3466 - val_loss: 0.3161
Epoch 6/20
500/500 [==============================] - 66s 131ms/step - loss: 0.3410 - val_loss: 0.3119
Epoch 7/20
500/500 [==============================] - 66s 131ms/step - loss: 0.3402 - val_loss: 0.3160
Epoch 8/20
500/500 [==============================] - 66s 131ms/step - loss: 0.3357 - val_loss: 0.3126
Epoch 9/20
500/500 [==============================] - 65s 131ms/step - loss: 0.3322 - val_loss: 0.3199
Epoch 10/20
500/500 [==============================] - 66s 132ms/step - l

In [33]:
model4 = models.Sequential()
model4.add(
    layers.GRU(32,
               dropout=0.1,
               recurrent_dropout=0.5,
               return_sequences=True,
               input_shape=(None, float_data.shape[-1])))
model4.add(
    layers.GRU(64, activation='relu', dropout=0.1, recurrent_dropout=0.5))
model4.add(layers.Dense(1))

In [34]:
model4.compile(optimizer=optimizers.rmsprop(), loss='mae')
history = model4.fit_generator(train_gen,
                               steps_per_epoch=500,
                               epochs=20,
                               validation_data=val_gen,
                               validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 181s 362ms/step - loss: 0.3843 - val_loss: 0.3377
Epoch 2/20
500/500 [==============================] - 178s 356ms/step - loss: 0.3599 - val_loss: 0.3266
Epoch 3/20
500/500 [==============================] - 176s 351ms/step - loss: 0.3567 - val_loss: 0.3148
Epoch 4/20
500/500 [==============================] - 176s 351ms/step - loss: 0.3484 - val_loss: 0.3214
Epoch 5/20
500/500 [==============================] - 176s 352ms/step - loss: 0.3451 - val_loss: 0.3415
Epoch 6/20
500/500 [==============================] - 176s 352ms/step - loss: 0.3402 - val_loss: 0.3105
Epoch 7/20
500/500 [==============================] - 176s 353ms/step - loss: 0.3376 - val_loss: 0.3194
Epoch 8/20
500/500 [==============================] - 176s 352ms/step - loss: 0.3347 - val_loss: 0.3185
Epoch 9/20
500/500 [==============================] - 176s 352ms/step - loss: 0.3325 - val_loss: 0.3274
Epoch 10/20
500/500 [==============================] - 176s 352m

reverse ltsm imdb

In [36]:
from keras.datasets import imdb

In [38]:
max_features = 10000
maxlen = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

In [40]:
type(x_train[0])

list

In [39]:
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [41]:
x_train = [x[::-1] for x in x_train]
x_test = [x[::-1] for x in x_test]

In [42]:
from keras.preprocessing import sequence

In [43]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [44]:
model5 = models.Sequential()
model5.add(layers.Embedding(max_features, 128))
model5.add(layers.LSTM(32))
model5.add(layers.Dense(1, activation='sigmoid'))

In [46]:
model5.compile(optimizer='rmsprop',
               loss='binary_crossentropy',
               metrics=['acc'])

In [47]:
history5 = model5.fit(x_train,
                      y_train,
                      epochs=10,
                      batch_size=128,
                      validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 71s 4ms/step - loss: 0.4832 - acc: 0.7690 - val_loss: 0.3547 - val_acc: 0.8644
Epoch 2/10
20000/20000 [==============================] - 71s 4ms/step - loss: 0.3105 - acc: 0.8824 - val_loss: 0.3705 - val_acc: 0.8490
Epoch 3/10
20000/20000 [==============================] - 76s 4ms/step - loss: 0.2546 - acc: 0.9057 - val_loss: 0.3321 - val_acc: 0.8586
Epoch 4/10
20000/20000 [==============================] - 78s 4ms/step - loss: 0.2182 - acc: 0.9208 - val_loss: 0.4067 - val_acc: 0.8642
Epoch 5/10
20000/20000 [==============================] - 74s 4ms/step - loss: 0.1973 - acc: 0.9306 - val_loss: 0.3504 - val_acc: 0.8668
Epoch 6/10
20000/20000 [==============================] - 74s 4ms/step - loss: 0.1734 - acc: 0.9398 - val_loss: 0.3842 - val_acc: 0.8746
Epoch 7/10
20000/20000 [==============================] - 70s 4ms/step - loss: 0.1584 - acc: 0.9448 - val_loss: 0.4484 - val_acc:

In [ ]:
BidiRNN

In [50]:
def reverse_order_generator(
                            data,
                            lookback,
                            delay,
                            min_index,
                            max_index,
                            shuffle=False,
                            batch_size=128,
                            step=6):
    if max_index is None:
        max_index=len(data)-delay-1
    i=min_index+lookback
    
    while 1:
        if shuffle:
            rows=np.random.randint(min_index+lookback,max_index,step)
        else:
            if i+batch_size>max_index:
                i=min_index+lookback
            rows=np.arange(i,min(i+lookback,max_index))
        
        samples=np.zeros((len(rows),lookback//step,data.shape[-1]))
        targets=np.zeros((len(rows),))
        
        for j,row in enumerate(rows):
            indices=range(rows[j]-lookback,rows[j],step)
            samples[j]=data[indices]
            targets[j]=data[rows[j]+delay][1]
        yield samples[:,::-1,:],targets
            

In [51]:
train_gen_reverse = reverse_order_generator(float_data,
                                            lookback=lookback,
                                            delay=delay,
                                            min_index=0,
                                            max_index=200000,
                                            shuffle=True,
                                            step=step,
                                            batch_size=batch_size)
val_gen_reverse = reverse_order_generator(float_data,
                                          lookback=lookback,
                                          delay=delay,
                                          min_index=200001,
                                          max_index=300000,
                                          step=step,
                                          batch_size=batch_size)

In [53]:
model6 = models.Sequential()
model6.add(layers.Bidirectional(layers.GRU(32),input_shape=(None,float_data.shape[-1])))
model6.add(layers.Dense(1))

model6.compile(optimizer=optimizers.RMSprop(), loss='mae')
history6 = model6.fit_generator(train_gen_reverse,
                               steps_per_epoch=500,
                               epochs=20,
                               validation_data=val_gen_reverse,
                               validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 245s 490ms/step - loss: 0.3829 - val_loss: 0.3522
Epoch 2/20
500/500 [==============================] - 243s 486ms/step - loss: 0.3580 - val_loss: 0.3197
Epoch 3/20
500/500 [==============================] - 243s 486ms/step - loss: 0.3477 - val_loss: 0.3697
Epoch 4/20
500/500 [==============================] - 240s 479ms/step - loss: 0.3286 - val_loss: 0.2708
Epoch 5/20
500/500 [==============================] - 238s 477ms/step - loss: 0.3456 - val_loss: 0.3051
Epoch 6/20
500/500 [==============================] - 234s 468ms/step - loss: 0.3345 - val_loss: 0.2861
Epoch 7/20
500/500 [==============================] - 238s 476ms/step - loss: 0.3302 - val_loss: 0.3383
Epoch 8/20
500/500 [==============================] - 237s 475ms/step - loss: 0.3313 - val_loss: 0.2670
Epoch 9/20
500/500 [==============================] - 235s 471ms/step - loss: 0.3391 - val_loss: 0.2972
Epoch 10/20
500/500 [==============================] - 236s 473m